In [ ]:
import pickle
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_df = pd.read_csv('C:/Users/ylou/MLC-Spring 2024/MLC-Week13-16-Group Project-Order Forecast/rohlik-orders-forecasting-challenge/train.csv') ## Load the train dataset

In [ ]:
def add_date_features(df): ## Convert date features into numerical features
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
    df.drop('date', axis=1, inplace=True)
    return df

In [ ]:
train_df = add_date_features(train_df)

In [ ]:
cols_to_drop = ['shutdown', 'mini_shutdown', 'mov_change', 'frankfurt_shutdown', 'precipitation', 'snow', 'user_activity_1', 'user_activity_2'] 
train_df.drop(cols_to_drop, axis=1, inplace=True) ## Drop irrelevant features from train dataset 

In [ ]:
label_encoders = {} ## Apply label encoding on the warehouse and holiday_name features
for col in ['warehouse', 'holiday_name']:
    label_encoders[col] = LabelEncoder()
    train_df[col] = label_encoders[col].fit_transform(train_df[col])

In [ ]:
def ensure_numeric(df): ## Ensure all features are in numerical format for the model development and prediction
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = LabelEncoder().fit_transform(df[col])
    return df

In [ ]:
train_df = ensure_numeric(train_df)

In [ ]:
train_df.drop('id', axis=1, inplace=True) ## Remove "id" feature from train dataset
train_df.drop('blackout', axis=1, inplace=True) ## Remove "blackout" feature from train dataset

In [ ]:
train_df

In [ ]:
X = train_df.drop('orders', axis=1) ## The target variable is "orders" and remove it from the independent variables (X)
y = train_df['orders'] ## Define target variable as "orders" for y

In [ ]:
X

In [ ]:
y

In [ ]:
model = XGBRegressor()
model.fit(X, y)

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [1]:
from flask import Flask, jsonify, request, render_template
import json
import numpy as np
import pickle

In [2]:
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

In [3]:
app = Flask(__name__, template_folder='C:/ProgramData/anaconda3/Lib/site-packages/flask/templates')

In [4]:
@app.route('/', methods=["GET", "POST"])
def index():
    pred = ""
    if request.method == "POST":
        warehouse = request.form["warehouse"]
        holiday_name = request.form["holiday_name"]
        holiday = request.form["holiday"]
        shops_closed = request.form["shops_closed"]
        winter_school_holidays = request.form["winter_school_holidays"]
        school_holidays = request.form["school_holidays"]
        year = request.form["year"]
        month = request.form["month"]
        day = request.form["day"]
        dayofweek = request.form["dayofweek"]
        is_weekend = request.form["is_weekend"]
        X = np.array([[float(warehouse), float(holiday_name), float(holiday), float(shops_closed), float(winter_school_holidays), float(school_holidays), float(year), float(month), float(day), float(dayofweek), float(is_weekend)]])
        pred = model.predict(X)[0]
    return render_template("index_new.html", pred=pred)

In [ ]:
if __name__ == "__main__":
    app.run(debug=False, host='127.0.0.1', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Jul/2024 00:18:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2024 00:18:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2024 00:33:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2024 00:33:25] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2024 00:33:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2024 00:33:48] "POST / HTTP/1.1" 200 -
